
agrupar por cidade
agrupar por ano count total por cidade
valor de univercidade
valor total de univercidade
col: distric code
county(condado):ano -> col:awardee(univercidades(gerar contador))-> toltal recebido

Mantém a D_50 (é redundante, mas deixa la)
Cria d_59 que vai ter tudo 0 ai quando agregarmos ela vai receber o valor de quantas vezes uma mesma universidade apareceu e vc pode usar a coluna d_50 para contar (se quiser)
e uma outra coluna d_99 que vai receber também 0 no começo e quando agregar ela fica com 1,2,3,4... conforme aparecer a mesma universidade mais vezes com valores acima de 50k
No fim temos 3 colunas
d_50 que é uma dummy mesmo
d_59 que tem quantas vezes uma universidade apareceu por ano com valor de 50k, 1,2,3,4...
d_99 que tem quantas vezes uma universidade apareceu por ano com valor ACIMA de 50k, 1,2,3,4...


Para os outros fundings usa o valor total, mas conta quantos fundins recebeu


- JASON

Why we have I-Corps with 0 investments? Why we have 0 investment in Troy for the first year receiving grant (SBIR if I am not mistaken) 


In [16]:
import pandas as pd
import numpy as np
from pathlib import Path

#path append
#base = Path('C:/workspace/nsf-award-extract') # Vitor's PC
base = Path('C:/collaboration/vitor/NSF_DATA') # Server

nsf=pd.read_csv(base / 'allNSF1.csv',index_col = False) #Change this to the clean data later
#nsf = pd.read_csv('allNSF1.csv',low_memory = False)
#dft=pd.read_csv(base/ 'test.csv',index_col = False) #Change this to the clean data later

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\collaboration\\vitor\\NSF_DATA\\allNSF1.csv'

In [ ]:
# rename  values of fundProgram
nsf = nsf.replace('IUCRC-Indust-Univ Coop Res Ctr','IUCRC')
nsf = nsf.replace('PFI-Partnrships for Innovation','PFI')
nsf = nsf.replace('STTR Phase I','STTR_1')
nsf = nsf.replace('STTR Phase II','STTR_2')
nsf = nsf.replace('SBIR Phase I','SBIR_1')
nsf = nsf.replace('SBIR Phase II','SBIR_2')

In [ ]:
#Just checking if we have the same amount of register in each year as in this link https://www.nsf.gov/awardsearch/download.jsp
#nsf['year'] = pd.to_datetime(nsf['date'], format='%m/%d/%Y').dt.year

Filling the missing county

In [ ]:
mapping = nsf.groupby('awardee')['awardeeCounty'].first().to_dict()
nsf['awardeeCounty'] = nsf['awardeeCounty'].fillna(nsf['awardee'].map(mapping))

Keeping only the funding with fundProgramName specified

In [ ]:
nsf = nsf[~nsf['fundProgramName'].isna()]

# Extracting the Universities that has missing county
We need to use this list to find the county and later use the filled file to map this location back to the main file

In [ ]:
# ta = nsf[nsf['awardee'] == "RENSSELAER POLYTECHNIC INSTITUTE"]
# ta[ta['estimatedTotalAmt'].isnull()]

In [ ]:
nsf[nsf['awardeeCounty'].isna()]

,Column,abstractText,agency,awardAgencyCode,awardee,awardeeAddress,awardeeCity,awardeeCountryCode,awardeeCounty,awardeeDistrictCode,...,perfCounty,perfDistrictCode,perfStateCode,primaryProgram,coPDPI,poName,poPhone,publicationResearch,publicationConference,projectOutComesReport
8,7,Continuous aseptic processing and packaging is...,NSF,4900.0,NORTH CAROLINA STATE UNIVERSITY,2601 Wolf Village Way,Raleigh,US,None,NC04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,38,The Trek of the Mammoth is a three-week multid...,NSF,4900.0,CHADRON STATE COLLEGE,10th and Main Streets,chadron,US,None,NE03,...,NaN,NaN,NaN,NaN,Ronald R Weedon,NaN,NaN,NaN,NaN,NaN
385,55,This three-year project is designed to improve...,NSF,4900.0,MCNEESE STATE UNIVERSITY,Box 90655,Lake Charles,US,None,LA07,...,NaN,NaN,NaN,NaN,Gary D Kratzer,NaN,NaN,NaN,NaN,NaN
924,7,PROJECT PTRA-PLUS is a program of the American...,NSF,4900.0,AMERICAN ASSOCIATION OF PHYSICS TEACHERS INC,One Physics Ellipse,College Park,US,None,MD05,...,NaN,MD05,MD,"490106 NSF, EDUCATION & HUMAN RESOURC",James H Nelson,NaN,NaN,NaN,NaN,NaN
993,76,The Bermuda Biological Station will receive su...,NSF,4900.0,BERMUDA INSTITUTE OF OCEAN SCIENCES (BIOS) INC.,17 Biological Station,St. George's GE01,BD,None,NaN,...,NaN,NaN,NaN,490100 NSF RESEARCH & RELATED ACTIVIT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191110,11257,New cryptographic systems can improve the secu...,NSF,4900.0,NORTH CAROLINA STATE UNIVERSITY,2601 Wolf Village Way,Raleigh,US,None,NC04,...,RALEIGH,NC02,NC,NaN,NaN,Sankar Basu,7.032928e+09,NaN,NaN,NaN
191356,11503,The U.S. Southwest is a drought-prone region t...,NSF,4900.0,NaN,NaN,Ann Arbor,US,NaN,MI,...,DAVIS,CA04,CA,NaN,NaN,Soumaya Belmecheri,7.032929e+09,"Macarewich, Sophia I. and Poulsen, Christopher...",NaN,Water resource use in the southwestern U.S. (h...
191433,11580,Extreme weather events (heat waves and cold sn...,NSF,4900.0,NORTH CAROLINA STATE UNIVERSITY,2601 Wolf Village Way,Raleigh,US,None,NC04,...,Raleigh,NC02,NC,NaN,NaN,Bruce Hamilton,7.032920e+09,NaN,NaN,NaN
191476,11623,This award is funded in whole or in part under...,NSF,4900.0,NORTH CAROLINA STATE UNIVERSITY,2601 Wolf Village Way,Raleigh,US,None,NC04,...,RALEIGH,NC02,NC,NaN,NaN,Bianca Garner,7.032928e+09,NaN,NaN,NaN


In [ ]:
#missing_NSF = nsf[nsf['awardeeCounty'].isna()].drop_duplicates(subset='awardee')
#missing_NSF.to_csv(base/ "missing_NSF.csv", index = False)
filled_NSF = pd.read_csv(base / 'filled_NSF.csv')


Code for filling the missing value should come here

In [ ]:
# Create a mapping dictionary from 'filled_NSF' dataset
mapping = filled_NSF.set_index('awardee')['awardeeCounty'].to_dict()

# Fill missing 'awardeeCounty' values in 'nsf' dataset using the mapping
nsf['awardeeCounty'] = nsf['awardeeCounty'].fillna(nsf['awardee'].map(mapping))

# Updated 'nsf' dataset with filled 'awardeeCounty' values

In [ ]:
nsf[nsf['awardeeCounty'].isna()]


,Column,abstractText,agency,awardAgencyCode,awardee,awardeeAddress,awardeeCity,awardeeCountryCode,awardeeCounty,awardeeDistrictCode,...,perfCounty,perfDistrictCode,perfStateCode,primaryProgram,coPDPI,poName,poPhone,publicationResearch,publicationConference,projectOutComesReport


Creating the file with no missing values (later we don't need this because we will have all the Universities with their respective county)

In [ ]:
nsf_all = nsf[~nsf['awardeeCounty'].isna()] #187755 + 3773 (correct) 191528 



Creating the Panel with aggregation (I think we don't need this)

Creating the first panel

In [ ]:
# def cross_to_panel(df):
#     df = df.set_index(['Year_date','awardeeCounty'])
#     df = df.sort_index()
#     return df
# panel_data = cross_to_panel(nsf_all)
#panel_data.to_csv('panel2.csv')

# def cross_to_panel(df):
#     df['date'] = pd.to_datetime(df['Year_date'], format='%Y')
#     df['year'] = df['date'].dt.year
#     df = df.set_index(['year', 'awardeeCounty'])
#     return df

# # Example usage

# panel_data = cross_to_panel(df)
# panel_data.to_csv('cross2.csv')
#panel_data

In [ ]:
#grouped_multiple = df.groupby(['Year_date', 'awardeeCounty'])
#grouped_multiple = grouped_multiple.reset_index()
#print(grouped_multiple)

# p_table = pd.pivot_table(nsf_all, index = ['Year_date',"awardeeCounty"])
# p_table.to_csv("panel_nsf_all.csv")
# p_table

Creating Filters to get the correct programs

In [ ]:
# create new dataframes of slices fundProgram
df_corps = nsf_all[nsf_all['fundProgramName'] == 'I-Corps']

df_SBIR_1 = nsf_all[nsf_all['fundProgramName'] == 'SBIR_1']
df_SBIR_2 = nsf_all[nsf_all['fundProgramName'] == 'SBIR_2']
df_STTR_1= nsf_all[nsf_all['fundProgramName'] == 'STTR_1']
df_STTR_2 = nsf_all[nsf_all['fundProgramName'] == 'STTR_2']
df_PFI = nsf_all[nsf_all['fundProgramName'] == 'PFI']
df_IUCRC = nsf_all[nsf_all['fundProgramName'] == 'IUCRC']

#combine all the other ones

In [ ]:
df_PFI

,Column,abstractText,agency,awardAgencyCode,awardee,awardeeAddress,awardeeCity,awardeeCountryCode,awardeeCounty,awardeeDistrictCode,...,perfCounty,perfDistrictCode,perfStateCode,primaryProgram,coPDPI,poName,poPhone,publicationResearch,publicationConference,projectOutComesReport
47839,4166,0090523\t\t\nKohn\t\t\t\n\nThis award is to Ru...,NSF,4900.0,"RUTGERS, THE STATE UNIVERSITY OF NEW JERSEY",3 RUTGERS PLZA,NEW BRUNSWICK,US,NEW BRUNSWICK,NJ06,...,New Brunswick,NJ06,NJ,NaN,Bozena Michniak,NaN,NaN,NaN,NaN,NaN
50013,1100,0125449\t\nSilverman\t\t\t\n\nThis award is to...,NSF,4900.0,UNIVERSITY OF SOUTHERN CALIFORNIA,3720 S FLOWER ST FL 3,LOS ANGELES,US,Los Angeles,CA37,...,Los Angeles,CA37,CA,NaN,Kathleen Allen,NaN,NaN,NaN,NaN,NaN
50153,1240,0125752\t\nZiliaskopoulos\t\t\t\n\nThis award ...,NSF,4900.0,NORTHWESTERN UNIVERSITY,633 CLARK,EVANSTON,US,Evanston,IL07,...,Evanston,IL07,IL,NaN,Jay Franke,NaN,NaN,NaN,NaN,NaN
50154,1241,0125429\nCraig\t\n\t\nThis award is to Montana...,NSF,4900.0,MONTANA STATE UNIVERSITY,216 MONTANA HALL,BOZEMAN,US,Bozeman,MT00,...,NaN,MT00,MT,NaN,James A McMillan,NaN,NaN,NaN,NaN,NaN
50155,1242,0125343\t\nDagher\t\t\n\nThis award is to the ...,NSF,4900.0,UNIVERSITY OF MAINE SYSTEM,5717 CORBETT HALL RM 422,ORONO,US,Orono,ME02,...,Orono,ME02,ME,NaN,James S Ward,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191275,11422,The broader impact/commercial potential of thi...,NSF,4900.0,UNIVERSITY OF WASHINGTON,4333 Brooklyn Ave NE,Seattle,US,Seattle,WA07,...,SEATTLE,WA07,WA,NaN,NaN,Jesus Soriano Molla,7.032928e+09,NaN,NaN,NaN
191278,11425,The broader impact/commercial potential of thi...,NSF,4900.0,UNIVERSITY OF TEXAS AT AUSTIN,110 INNER CAMPUS DR,Austin,US,Austin,TX10,...,AUSTIN,TX25,TX,NaN,NaN,Jesus Soriano Molla,7.032928e+09,"Grose, Joshua and Dibua, Obehi G and Behera, D...",NaN,NaN
191279,11426,The broader impact/commercial potential of thi...,NSF,4900.0,REGENTS OF THE UNIVERSITY OF MICHIGAN,503 THOMPSON ST,Ann Arbor,US,Ann Arbor,MI12,...,Ann Arbor,MI12,MI,NaN,Richard T Zvosec,Jesus Soriano Molla,7.032928e+09,NaN,NaN,NaN
191280,11427,The broader impact/commercial potential of thi...,NSF,4900.0,REGENTS OF THE UNIVERSITY OF MICHIGAN,503 THOMPSON ST,Ann Arbor,US,Ann Arbor,MI12,...,Ann Arbor,MI12,MI,NaN,NaN,Jesus Soriano Molla,7.032928e+09,NaN,NaN,NaN


In [ ]:
df_SBIR_2

,Column,abstractText,agency,awardAgencyCode,awardee,awardeeAddress,awardeeCity,awardeeCountryCode,awardeeCounty,awardeeDistrictCode,...,perfCounty,perfDistrictCode,perfStateCode,primaryProgram,coPDPI,poName,poPhone,publicationResearch,publicationConference,projectOutComesReport
1009,92,This is an award for conduct of research the o...,NSF,4900.0,OUSD(AT&L),11 Industrial Way,Atkinson,US,Atkinson,NH02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426,509,During the first phase of this NSF sponsored S...,NSF,4900.0,OUSD(AT&L),11409 Valley View Road,EDEN PRAIRIE,US,Eden Prairie,MN03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1690,773,This Small Business Innovation Research (SBIR)...,NSF,4900.0,NaN,301 Chesapeake Drive,REDWOOD CITY,US,Redwood City,CA14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2862,1945,During the first phase of this NSF sponsored S...,NSF,4900.0,NaN,3453 Commercial Ave,NORTHBROOK,US,Northbrook,IL10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3269,39,This project is funded under the National Scie...,NSF,4900.0,OUSD(AT&L),3450 Hillview Avenue,Palo Alto,US,Palo Alto,CA18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191298,11445,The broader impact/commercial potential of thi...,NSF,4900.0,"HEALTH TECHNOLOGY INNOVATIONS, INC.",4640 SW MACADAM AVE STE 200D,Portland,US,Portland,OR01,...,PORTLAND,OR03,OR,NaN,NaN,Alastair Monk,7.032924e+09,NaN,NaN,NaN
191299,11446,The broader impact/commercial potential of thi...,NSF,4900.0,"AMBI ROBOTICS, INC.",1610 5TH ST,BERKELEY,US,Berkeley,CA09,...,EMERYVILLE,CA12,CA,NaN,NaN,Elizabeth Mirowski,7.032923e+09,NaN,NaN,NaN
191424,11571,The broader impact/commercial potential of thi...,NSF,4900.0,"PSYONIC, INC.",60 HAZELWOOD DR STE 210,CHAMPAIGN,US,Champaign,IL13,...,CHAMPAIGN,IL13,IL,NaN,NaN,Alastair Monk,7.032924e+09,NaN,NaN,NaN
191499,11646,The broader impact/commercial potential of thi...,NSF,4900.0,"CODON LEARNING, INC.",607 10TH ST,GOLDEN,US,Golden,CO07,...,GOLDEN,CO07,CO,NaN,NaN,Rajesh Mehta,7.032922e+09,NaN,NaN,NaN


Now we have to deal with I-Coprs before we continue with the code

# Repeating the same process we did for I-Corps

In [ ]:
#substituir data por df_corps
df_corps['d_inv'] = df_corps.apply(lambda x: 0 if x.estimatedTotalAmt <= 50000.0 else 1,axis = 1) # create the variable 50k < and 50k >
df_corps['d_59'] = 1
df_corps['d_99'] = 0
df_corps['estimatedTotalAmt_d_99'] = 0


C:\Users\fernav4\AppData\Local\Temp\ipykernel_15352\2879990113.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corps['d_inv'] = df_corps.apply(lambda x: 0 if x.estimatedTotalAmt <= 50000.0 else 1,axis = 1) # create the variable 50k < and 50k >
C:\Users\fernav4\AppData\Local\Temp\ipykernel_15352\2879990113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corps['d_59'] = 1
C:\Users\fernav4\AppData\Local\Temp\ipykernel_15352\2879990113.py:4: SettingWithCopyWarning: 
A value is trying to be set on 

In [ ]:
df_clear = df_corps.groupby(['awardeeCounty','Year_date','awardee','d_inv'])[['d_59','estimatedTotalAmt','d_99','estimatedTotalAmt_d_99']].sum()# agrupando por county
df_clear = df_clear.reset_index()

df_below50 = df_clear[df_clear['d_inv'] == 0].reset_index()
df_below50 = df_below50.drop(columns =['index'])

df_over50 = df_clear[df_clear['d_inv'] == 1].reset_index()
df_over50['d_99'] = df_over50['d_59']
df_over50['d_59'] = 0
df_over50['estimatedTotalAmt_d_99'] = df_over50['estimatedTotalAmt']
df_over50['estimatedTotalAmt'] = 0
df_over50 = df_over50.drop(columns = ['index'])


In [ ]:
for index,row in df_below50.iterrows():
  key = row['awardee']
  met = df_over50[df_over50['awardee'] == key]
  key_year = row['Year_date']
  met_year = met[met['Year_date'] == key_year]
  if met_year.shape[0]>0:
    df_below50.at[index,'d_99'] = met_year.iloc[0]['d_99']
    df_below50.at[index,'estimatedTotalAmt_d_99'] = met_year.iloc[0]['estimatedTotalAmt_d_99']
    df_over50 = df_over50.drop(met_year.index[0])

In [ ]:
df_below50 = df_below50.merge(df_over50,how = 'outer')
df_below50['total_corps'] = df_below50.apply(lambda x: x['estimatedTotalAmt'] + x['estimatedTotalAmt_d_99'],axis = 1)
df_corps_f = df_below50.sort_values(by = ['awardeeCounty','Year_date'])
df_corps_f = df_corps_f.reset_index(inplace = False).drop(columns = ['index'])


In [ ]:
#df_corps_f.loc[df_corps_f['awardeeCounty'] == 'Troy']

In [ ]:
df_corps_county = df_corps_f.groupby(['awardeeCounty','Year_date','d_inv'])[['d_59','estimatedTotalAmt','d_99','estimatedTotalAmt_d_99', 'total_corps']].sum()# agrupando por county
df_corps_county= df_corps_county.reset_index()
df_corps_county

,awardeeCounty,Year_date,d_inv,d_59,estimatedTotalAmt,d_99,estimatedTotalAmt_d_99,total_corps
0,Akron,2012,0,1,50000.0,0,0.0,50000.0
1,Akron,2014,0,1,50000.0,0,0.0,50000.0
2,Akron,2016,0,2,100000.0,0,0.0,100000.0
3,Akron,2017,0,2,100000.0,0,0.0,100000.0
4,Akron,2018,0,1,50000.0,0,0.0,50000.0
...,...,...,...,...,...,...,...,...
757,Worcester,2020,0,2,100000.0,0,0.0,100000.0
758,Worcester,2021,0,4,200000.0,0,0.0,200000.0
759,Worcester,2022,0,1,50000.0,0,0.0,50000.0
760,Youngstown,2016,0,1,50000.0,0,0.0,50000.0


In [ ]:
#df_corps_county.loc[(df_corps_county['d_59'] == 1) & (df_corps_county['d_99'] > 0)]


In [ ]:
#df_corps_county.loc[df_corps_county['d_99'] == 1]

Saving the file

In [ ]:
# df_final
# df_final.to_csv("nsf_all.csv", index=False)

Back with the DataFrame

In [ ]:
# # separating the fundings
# df_others2 = nsf_all.copy()
# for index,rows in nsf_all.iterrows():
#     if rows.fundProgramName  in ['IUCRC','PFI','STTR_1','STTR_2','SBIR_1','SBIR_2','I-Corps']:
#         df_others2.drop(index = index, inplace = True) 
#     else:
#         pass

In [ ]:
#df_others2[df_others2['fundProgramName'] == 'I-Corps']
#df_others2.to_csv(base/'df_others2.csv', index=False)

df_others2 = pd.read_csv(base/'df_others2.csv')

C:\Users\fernav4\AppData\Local\Temp\ipykernel_15352\2390504702.py:4: DtypeWarning: Columns (12,17,32,37,45,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df_others2 = pd.read_csv(base/'df_others2.csv')


In [ ]:
df_others2.shape #181150


(184718, 52)

In [ ]:
# incluir coluna

nsf.loc[nsf['awardeeCounty'] == "0"]

,Column,abstractText,agency,awardAgencyCode,awardee,awardeeAddress,awardeeCity,awardeeCountryCode,awardeeCounty,awardeeDistrictCode,...,perfCounty,perfDistrictCode,perfStateCode,primaryProgram,coPDPI,poName,poPhone,publicationResearch,publicationConference,projectOutComesReport


In [ ]:
def number_of_invest(data,name):
    data[f'd_inv_{str(name)}'] = data.apply(lambda x: 0 if x.estimatedTotalAmt == 0 else 1,axis = 1)# cria variavel categorica para valores iguais 0 ou maior que 0
    df_clear = data.groupby(['awardeeCounty', 'Year_date' , f'd_inv_{str(name)}'])[['estimatedTotalAmt']].sum()# agrupando 
    df_clear = df_clear.rename(columns={'estimatedTotalAmt':f'{name}'})
    df_clear = df_clear.reset_index()
    return df_clear

In [ ]:
x = number_of_invest(df_SBIR_1,"df_SBIR_1")
y = number_of_invest(df_SBIR_2,"df_SBIR_2")
z = number_of_invest(df_STTR_1,"df_STTR_1")
k = number_of_invest(df_STTR_2,"df_STTR_2")
i = number_of_invest(df_PFI,"df_PFI")
l = number_of_invest(df_others2,"df_others")

C:\Users\fernav4\AppData\Local\Temp\ipykernel_15352\3191795940.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'd_inv_{str(name)}'] = data.apply(lambda x: 0 if x.estimatedTotalAmt == 0 else 1,axis = 1)# cria variavel categorica para valores iguais 0 ou maior que 0
C:\Users\fernav4\AppData\Local\Temp\ipykernel_15352\3191795940.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'd_inv_{str(name)}'] = data.apply(lambda x: 0 if x.estimatedTotalAmt == 0 else 1,axis = 1)# cria variavel categorica

In [ ]:
x

,awardeeCounty,Year_date,d_inv_df_SBIR_1,df_SBIR_1
0,ABINGDON,2021,1,256000.0
1,ADDISON,2022,1,254214.0
2,ALAMO,2021,1,256000.0
3,AMBLER,2020,1,419610.0
4,AMESBURY,2020,1,254504.0
...,...,...,...,...
1232,Yavapai County,2009,1,0.0
1233,Yorktown Heights,2019,1,225000.0
1234,Ypsilanti,2012,1,149960.0
1235,Ypsilanti,2019,1,215181.0


In [ ]:
df_teste = x.merge(y,how = "outer")

#df_teste = df_teste.merge(m,how = 'outer')

In [ ]:
df_teste = df_teste.merge(z,how = "outer")
df_teste = df_teste.merge(k,how = "outer")
df_teste = df_teste.merge(i,how = "outer")
df_teste = df_teste.merge(l,how = "outer")

In [ ]:
df_teste #24407 

,awardeeCounty,Year_date,d_inv_df_SBIR_1,df_SBIR_1,d_inv_df_SBIR_2,df_SBIR_2,d_inv_df_STTR_1,df_STTR_1,d_inv_df_STTR_2,df_STTR_2,d_inv_df_PFI,df_PFI,d_inv_df_others,df_others
0,ABINGDON,2021,1.0,256000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADDISON,2022,1.0,254214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALAMO,2021,1.0,256000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AMBLER,2020,1.0,419610.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AMESBURY,2020,1.0,254504.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16087,Yuma,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1200000.0
16088,Zanesville,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,818277.0
16089,Zanesville,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,487868.0
16090,Zionsville,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,302534.0


In [ ]:
df_nsf = df_teste.merge(df_corps_county,how = 'outer')

In [ ]:
def convert_to_int(value):
    try:
        return int(value)
    except ValueError:
        return value

df_nsf = df_nsf.fillna(0)  # Fill NaN values with zeros
df_nsf = df_nsf.applymap(convert_to_int)  # Apply conversion function to each element


In [ ]:
df_nsf #22873 

,awardeeCounty,Year_date,d_inv_df_SBIR_1,df_SBIR_1,d_inv_df_SBIR_2,df_SBIR_2,d_inv_df_STTR_1,df_STTR_1,d_inv_df_STTR_2,df_STTR_2,d_inv_df_PFI,df_PFI,d_inv_df_others,df_others,d_inv,d_59,estimatedTotalAmt,d_99,estimatedTotalAmt_d_99,total_corps
0,ABINGDON,2021,1,256000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ADDISON,2022,1,254214,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ALAMO,2021,1,256000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,AMBLER,2020,1,419610,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,AMESBURY,2020,1,254504,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16106,Mequon,2019,0,0,0,0,0,0,0,0,0,0,0,0,0,1,50000,0,0,50000
16107,Ponce,2017,0,0,0,0,0,0,0,0,0,0,0,0,0,1,50000,0,0,50000
16108,Toledo,2016,0,0,0,0,0,0,0,0,0,0,0,0,0,6,300000,0,0,300000
16109,Valdosta,2015,0,0,0,0,0,0,0,0,0,0,0,0,0,1,50000,0,0,50000


In [ ]:
df_nsf.columns

Index(['awardeeCounty', 'Year_date', 'd_inv_df_SBIR_1', 'df_SBIR_1',
       'd_inv_df_SBIR_2', 'df_SBIR_2', 'd_inv_df_STTR_1', 'df_STTR_1',
       'd_inv_df_STTR_2', 'df_STTR_2', 'd_inv_df_PFI', 'df_PFI',
       'd_inv_df_others', 'df_others', 'd_inv', 'd_59', 'estimatedTotalAmt',
       'd_99', 'estimatedTotalAmt_d_99', 'total_corps'],
      dtype='object')

In [ ]:
#df_teste['total_inv'] = df_teste.apply(lambda x: x['d_inv_df_SBIR_1']+ x['d_inv_df_SBIR_2']+x['d_inv_df_STTR_1']+x['d_inv_df_STTR_2']+x['d_inv_df_PFI']+x['d_inv_df_others'],axis=1)
df_nsf['total_fund'] = df_nsf.apply(lambda x: x['df_SBIR_1']+ x['df_SBIR_2']+x['df_STTR_1']+x['df_STTR_2']+x['df_PFI']+x['df_others']+x['total_corps'],axis=1)


In [ ]:
df_nsf.shape

(16111, 21)

In [ ]:
df_nsf

,awardeeCounty,Year_date,d_inv_df_SBIR_1,df_SBIR_1,d_inv_df_SBIR_2,df_SBIR_2,d_inv_df_STTR_1,df_STTR_1,d_inv_df_STTR_2,df_STTR_2,...,df_PFI,d_inv_df_others,df_others,d_inv,d_59,estimatedTotalAmt,d_99,estimatedTotalAmt_d_99,total_corps,total_fund
0,ABINGDON,2021,1,256000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,256000
1,ADDISON,2022,1,254214,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,254214
2,ALAMO,2021,1,256000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,256000
3,AMBLER,2020,1,419610,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,419610
4,AMESBURY,2020,1,254504,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,254504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16106,Mequon,2019,0,0,0,0,0,0,0,0,...,0,0,0,0,1,50000,0,0,50000,50000
16107,Ponce,2017,0,0,0,0,0,0,0,0,...,0,0,0,0,1,50000,0,0,50000,50000
16108,Toledo,2016,0,0,0,0,0,0,0,0,...,0,0,0,0,6,300000,0,0,300000,300000
16109,Valdosta,2015,0,0,0,0,0,0,0,0,...,0,0,0,0,1,50000,0,0,50000,50000


In [4]:
# rename columns
df_nsf = df_nsf.rename(columns={'df_SBIR_1':'SBIR_1','df_SBIR_2':'SBIR_2','df_STTR_1':'STTR_1','df_STTR_2':'STTR_2','df_PFI':'PFI','df_others':'others','d_inv_df_SBIR_1': 'd_SBIR_1', 'd_inv_df_SBIR_2':'d_SBIR_2', 'd_inv_df_STTR_1':'d_STTR_1', 'd_inv_df_STTR_2':'d_STTR_2', 'd_inv_df_PFI':'d_PFI', 'd_inv_df_others':'d_others', "estimatedTotalAmt" : "total_50k"})  # Rename columns 


In [1]:
df_nsf = df_nsf.sort_values(by = ['Year_date'])
df_nsf.to_csv(base/ "panel.csv", index=False)


NameError: name 'df_nsf' is not defined

# Preparing the Panel to run DID R Package 

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
df_nsf = pd.read_csv("panel.csv")

In [3]:
df_nsf.columns

Index(['awardeeCounty', 'Year_date', 'd_SBIR_1', 'SBIR_1', 'd_SBIR_2',
       'SBIR_2', 'd_STTR_1', 'STTR_1', 'd_STTR_2', 'STTR_2', 'd_PFI', 'PFI',
       'd_others', 'others', 'd_inv', 'd_59', 'total_50k', 'd_99',
       'estimatedTotalAmt_d_99', 'total_corps', 'total_fund'],
      dtype='object')

Create dummy 

In [4]:
def create_dummy_variables(df, columns_to_analyze, unit_name_col, year_col):
    # Make a copy of the original DataFrame to avoid modifying it directly
    df_copy = df.copy()
    
    # Sort the DataFrame by unit_name and year for accurate analysis
    df_copy.sort_values(by=[unit_name_col, year_col], inplace=True)
    
    # Loop through each column in the list of columns to analyze
    for col in columns_to_analyze:
        # Create a new column with the prefix 'd_' and the name of the column being evaluated
        dummy_col_name = f'd_{col}'
        df_copy[dummy_col_name] = 0  # Initialize all values to 0
        
        # Keep track of treated units and their treatment status
        treated_units = {}
        
        # Loop through each row to check for treatment and set dummy variable accordingly
        for _, row in df_copy.iterrows():
            unit = row[unit_name_col]
            year = row[year_col]
            value = row[col]
            
            # If the unit hasn't been treated yet and the value in the current column is not 0
            if unit not in treated_units and value != 0:
                treated_units[unit] = True
            
            # If the unit has been treated or already encountered a non-zero value before
            if unit in treated_units:
                df_copy.at[_, dummy_col_name] = 1
    
    return df_copy


Just testing the function

In [5]:
# Sample data
data = {
    'unit_name': ['A', 'A', 'B', 'B', 'B', 'C', 'C', 'C'],
    'year': [2000, 2001, 2000, 2001, 2002, 2000, 2001, 2002],
    'col1': [1, 0, 2, 3, 0, 0, 0, 0],
    'col2': [0, 0, 0, 4, 0, 1, 0, 0],
    'col3': [5, 0, 0, 0, 0, 0, 6, 0]
}

# Create the DataFrame
df = pd.DataFrame(data)

# List of columns for analysis
columns_list = ['col1', 'col2', 'col3']

# Call the function to create dummy variables
result_df = create_dummy_variables(df, columns_list, 'unit_name', 'year')

# Display the resulting DataFrame
print(result_df)


  unit_name  year  col1  col2  col3  d_col1  d_col2  d_col3
0         A  2000     1     0     5       1       0       1
1         A  2001     0     0     0       1       0       1
2         B  2000     2     0     0       1       0       0
3         B  2001     3     4     0       1       1       0
4         B  2002     0     0     0       1       1       0
5         C  2000     0     1     0       0       1       0
6         C  2001     0     0     6       0       1       1
7         C  2002     0     0     0       0       1       1


In [6]:
def convert_to_numeric_id(df, name_column):
    # Create a mapping of unique names to numeric IDs
    name_to_id = {name: i for i, name in enumerate(df[name_column].unique())}
    
    # Create a new column in the DataFrame to store the numeric IDs
    df['ID'] = df[name_column].map(name_to_id)
    
    return df


For DID the dummy must actually have the year when first treated

In [7]:
df

,unit_name,year,col1,col2,col3
0,A,2000,1,0,5
1,A,2001,0,0,0
2,B,2000,2,0,0
3,B,2001,3,4,0
4,B,2002,0,0,0
5,C,2000,0,1,0
6,C,2001,0,0,6
7,C,2002,0,0,0


Create a function that receives these paramenters create_gname_columns(df, columns_list, id_col, year_col):

df: the dataframe that will be used
columns_list: the list of columns that will be used to verify if the treatment happened
id_col: the column that has the id
year_col: the column that has the year that can be used to check in which year the treatment happened or not

The treatment condition should be tested for each ID each time.
The function should return the dataframe with the new column g_(name of the column tested). I want to find who is being treated, being treated means that the column in the column list has any value different than zero. If that is the case then the year when the first treatment happened should be the value in the g_(column), if not then it should be zero. that should be mapping back to all lines for the same id.

Should use map function to map back the values to the corresponding column 

In [8]:
def create_gname_columns(df, columns_list, id_col, year_col):
    # Create a copy of the dataframe to avoid modifying the original one
    df_result = df.copy()

    # Initialize a dictionary to store the treatment year for each ID and column
    id_col_treatment_year = {}

    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Get the ID and year values from the current row
        current_id = row[id_col]
        current_year = row[year_col]

        # Check if the ID is already in the treatment year dictionary
        if current_id not in id_col_treatment_year:
            # If not, initialize a dictionary to store the treatment years for each column
            id_col_treatment_year[current_id] = {col: 0 for col in columns_list}

        # Check if any of the columns in the columns_list have a value different than zero
        for col in columns_list:
            if row[col] != 0:
                # Update the treatment year for the corresponding column if it's the first treatment year found
                if id_col_treatment_year[current_id][col] == 0:
                    id_col_treatment_year[current_id][col] = current_year

    # Create a dictionary to store the mapping from column name to the new 'g_(column)' name
    gname_mapping = {col: f'g_{col}' for col in columns_list}

    # Use the map function to create the new columns
    for col in columns_list:
        new_col_name = gname_mapping[col]
        df_result[new_col_name] = df_result[id_col].map(lambda x: id_col_treatment_year[x][col])

    return df_result


In [9]:
result_df = create_dummy_variables(df,  columns_list, 'unit_name', 'year')
result_df = convert_to_numeric_id(result_df,'unit_name')
result_df = create_gname_columns(result_df,  columns_list, 'ID', 'year')
# Display the resulting DataFrame
print(result_df)

  unit_name  year  col1  col2  col3  d_col1  d_col2  d_col3  ID  g_col1  \
0         A  2000     1     0     5       1       0       1   0    2000   
1         A  2001     0     0     0       1       0       1   0    2000   
2         B  2000     2     0     0       1       0       0   1    2000   
3         B  2001     3     4     0       1       1       0   1    2000   
4         B  2002     0     0     0       1       1       0   1    2000   
5         C  2000     0     1     0       0       1       0   2       0   
6         C  2001     0     0     6       0       1       1   2       0   
7         C  2002     0     0     0       0       1       1   2       0   

   g_col2  g_col3  
0       0    2000  
1       0    2000  
2    2001       0  
3    2001       0  
4    2001       0  
5    2000    2001  
6    2000    2001  
7    2000    2001  


Calling the functions to create the DF for DID package (we should also implement Sun and Abraham )

In [10]:
nsf_columns = ['SBIR_1', 'SBIR_2', 'STTR_1', 'STTR_2', 'PFI', 'others', 'd_59', ]

df_nsf = create_dummy_variables(df_nsf, nsf_columns, 'awardeeCounty', 'Year_date')

df_nsf = convert_to_numeric_id(df_nsf, 'awardeeCounty')

df_nsf = create_gname_columns(df_nsf, nsf_columns, 'awardeeCounty', 'Year_date')

In [11]:
df_nsf

,awardeeCounty,Year_date,d_SBIR_1,SBIR_1,d_SBIR_2,SBIR_2,d_STTR_1,STTR_1,d_STTR_2,STTR_2,...,total_fund,d_d_59,ID,g_SBIR_1,g_SBIR_2,g_STTR_1,g_STTR_2,g_PFI,g_others,g_d_59
14936,ABINGDON,2021,1,256000,0,0,0,0,0,0,...,256000,0,0,2021,0,0,0,0,0,0
9063,ADA,2010,0,0,0,0,0,0,0,0,...,63062,0,1,0,0,0,0,0,2010,0
10553,ADA,2013,0,0,0,0,0,0,0,0,...,103856,0,1,0,0,0,0,0,2010,0
13256,ADA,2018,0,0,0,0,0,0,0,0,...,2399334,0,1,0,0,0,0,0,2010,0
14573,ADA,2020,0,0,0,0,0,0,0,0,...,130333,0,1,0,0,0,0,0,2010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14050,Zanesville,2019,0,0,0,0,0,0,0,0,...,487868,0,2210,0,0,0,0,0,1998,0
176,Zionsville,1990,0,0,0,0,0,0,0,0,...,302534,0,2211,2019,2022,0,0,0,1990,0
1916,Zionsville,1994,0,0,0,0,0,0,0,0,...,1168219,0,2211,2019,2022,0,0,0,1990,0
13555,Zionsville,2019,1,449746,0,0,0,0,0,0,...,449746,0,2211,2019,2022,0,0,0,1990,0


In [12]:
df_nsf.to_csv("panel.csv", index=False)

# END
Previous code I-Corps

In [40]:
# pd.set_option('display.max_columns', None)
# df=pd.read_csv('genICorp.csv')
# dfp=pd.read_csv('Panel2.csv') #using the already organized data
# dfc = pd.read_csv("I-Corp_county.csv") # This is the file I manually updated the county column

# #File to create the panel
# dfp = pd.read_csv("I-corp-Full.csv")

In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! abaixo so testes !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
x = data[data['awardee']== 'REGENTS OF THE UNIVERSITY OF CALIFORNIA, THE']
len(x[x['Year_date']== 2017])

In [ ]:
df_below50[df_below50['awardee'] == 'NATIONAL COLLEGIATE INVENTORS & INNOVATORS ALLIANCE, INC.']

In [ ]:
df_below50[df_below50['awardee'] == 'THE UNIVERSITY OF AKRON']

In [ ]:
df_over50[df_over50['awardee'] == 'NATIONAL COLLEGIATE INVENTORS & INNOVATORS ALLIANCE, INC.']

In [ ]:
df_final[df_final['awardee'] == 'NATIONAL COLLEGIATE INVENTORS & INNOVATORS ALLIANCE, INC.']

In [ ]:
#df_melt = pd.melt(data,id_vars= ['Year_date','awardee','estimatedTotalAmt'], value_vars = ['d_inv'] )
#df_melt

In [ ]:
data[data['awardee'] == 'NATIONAL COLLEGIATE INVENTORS & INNOVATORS ALLIANCE, INC.']

In [ ]:
df_50 = df_50.groupby(['Year_date','awardee'])[['estimatedTotalAmt']].sum()# agrupando por county

In [ ]:
df_50 = df_50.reset_index( )


In [ ]:
SELECT * FROM `post` WHERE source_id >= 175 AND created_date < '2023-01-01'
85840

DELETE epecificar a tabela para apagar os dados FROM `insight` INNER JOIN `post` on insight.post_id = post.id WHERE post.source_id = 175 AND post.created_date < '2023-01-01';
 numeros de registros apagados 13377784

WITH postIds AS (
 SELECT id FROM post WHERE source_id >= 175 AND created_date < '2023-01-01'
)

DELETE *
FROM insight
WHERE post_id IN postIds

In [ ]:
79+1125

In [ ]:
df_over50.shape


In [ ]:
x = 0
for index,row in df_below50.iterrows():
  key = row['awardee']
  met = df_over50[df_over50['awardee'] == key]
  key_year = row['Year_date']
  met_year = met[met['Year_date'] == key_year]
  if met_year.shape[0] > 0:
    x+=1
    print(met_year.index[0])
print(x)

In [ ]:
dif = df_over50[df_over50['awardee'] != key]

In [ ]:
x = df_over50['awardee'] != df_below50['awardee']

In [ ]:
dif

,index,awardeeCounty,Year_date,awardee,d_inv,estimatedTotalAmt,d_59,d_99
0,1,Akron,2013,THE UNIVERSITY OF AKRON,1,300000,1,0
1,6,Akron,2017,THE UNIVERSITY OF AKRON,1,300000,1,0
2,18,Albuquerque,2017,"UNIVERSITY OF NEW MEXICO, THE",1,443631,1,0
3,22,Alexandria,2019,GEM CONSORTIUM,1,3498254,1,0
4,29,Ames,2017,IOWA STATE UNIVERSITY OF SCIENCE AND TECHNOLOGY,1,498750,1,0
...,...,...,...,...,...,...,...,...
163,1254,Washington,2017,"GEORGE WASHINGTON UNIVERSITY, THE",1,499502,1,0
164,1265,West Lafayette,2015,PURDUE UNIVERSITY,1,197280,1,0
165,1269,West Lafayette,2018,PURDUE UNIVERSITY,1,160000,1,0
166,1275,Wichita,2016,WICHITA STATE UNIVERSITY,1,200000,1,0


In [ ]:
df_test = data.groupby(['awardeeCounty','Year_date','awardee'])[['estimatedTotalAmt']].sum()# agrupando por county

In [ ]:
df_test

In [ ]:
#min_value = data.estimatedTotalAmt.min() # menor valor da coluna cfdaNumber
df_50['n_invest'] = df_50.apply(lambda x: x.estimatedTotalAmt/min_value if x.estimatedTotalAmt> min_value else min_value/min_value, axis=1)# add col n

In [ ]:
df_test.reset_index(inplace =True)
df_test.head(4)


In [ ]:
for index, row in df_test.iterrows():
  key = row['awardee']
  met = data[data['awardee']==key]


